In [1]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

from toy_samples import gaussian_samples
from covariance import logX_mu, logX_covinv_chol, points_at_iteration, logPr

# Load test samples
from tools import pickle_in
samples_g_1, samples_c_1, samples_w_1 = pickle_in("test_samples/samples_1.pickle")

#### Toy problem where underlying X distribution is exactly a multivariate Gaussian

Test whether approach to maximise logPr gives a better answer than least squares - it does. Tried for y = ax^2 and y = log(ax)

In [152]:
def log_func(x, theta):
    return np.log(theta*x)

def log_inverse(y, theta):
    return 1/theta * np.exp(y)


def linear_func(x, theta):
    return theta * x

def linear_inverse(y, theta):
    return y / theta

def logL_func(logX, theta):
    d = theta
    X = np.exp(logX)
    return - X**(2/d)

def logX_func(logL, theta):
    d = theta
    return d/2 * np.log(-logL)

In [247]:
from scipy.optimize import minimize
def optimise_pr_general(y, inverse, mean, cov_inv, x0):
    def negative_prob(theta):
        x = inverse(y, theta)
        return (x - mean).T @ cov_inv @ (x - mean) # want to maximise probability <-> minimise negative
    solution = minimize(negative_prob, x0)
    return solution

In [221]:
def optimise_repeat(mean, cov, func, funcinverse, theta_actual, theta0, covinv=None, repeats=10):
    if covinv is None:
        covinv = np.linalg.inv(cov)
    cg = []
    ls = []
    for i in range(repeats):
        x_actual = np.random.multivariate_normal(mean, cov)
        y = func(x_actual, theta_actual)
        a_cg, = optimise_pr_general(y, funcinverse, mean, covinv, theta0).x
        a_ls, = optimise_pr_general(y, funcinverse, mean, np.eye(len(mean)), theta0).x
        cg.append(a_cg)
        ls.append(a_ls)
    return np.array(cg), np.array(ls)

def optimise_repeat_stats(cg, ls):
    print(f"CG: {cg.mean():.4f} +/- {cg.std():.4f}")
    print(f"LS: {ls.mean():.2f} +/- {ls.std():.2f}")

CG does better for random covariances; the larger the covariance, the better it does

In [256]:
from covariance import points_at_iteration


      weights     
1000  8.377862e-11    500
1001  8.389654e-11    499
1002  8.632691e-11    498
1003  8.707837e-11    497
1004  8.833418e-11    496
                     ... 
1495  2.368732e-02      5
1496  5.353004e-02      4
1497  6.201663e-02      3
1498  5.825334e-01      2
1499  1.000000e+00      1
Name: nlive, Length: 500, dtype: int64

In [258]:
points = 100
mean = np.linspace(100, 200, points)
L = np.random.rand(points, points)
covrand = L @ L.T
covrand = covrand * 1e-4
covinvrand = np.linalg.inv(covrand)

nk = np.array(points_at_iteration(samples_g_1, 1000)[1000:].nlive)
mean_ns = logX_mu(nk)
covinv_ns = logX_covinv_chol(nk)
cov_ns = np.linalg.inv(covinv_ns)

In [253]:
# Linear function
repeats = optimise_repeat(mean_ns, covrand*1e-2, linear_func, linear_inverse, theta_actual=10, theta0=10, covinv=covinvrand, repeats=10)
optimise_repeat_stats(*repeats)

CG: 9.9999 +/- 0.0072
LS: 10.29 +/- 0.49


In [241]:
# Log function
repeats = optimise_repeat(mean, covrand*1e6, log_func, log_inverse, theta_actual=10, theta0=10, repeats=10)
optimise_repeat_stats(*repeats)

/tmp/ipykernel_73027/3236866486.py:2: RuntimeWarning: invalid value encountered in log
  return np.log(theta*x)


CG: 10.1257 +/- 0.0921
LS: 10.36 +/- 1.15


CG does worse for NS covariance matrix

In [259]:
# Using NS mean and covariance matrices
repeats = optimise_repeat(mean_ns, cov_ns, logL_func, logX_func, theta_actual=10, theta0=10, covinv=covinv_ns, repeats=10)
optimise_repeat_stats(*repeats)

CG: 4.9798 +/- 0.1574
LS: 10.24 +/- 0.56


In [246]:
# Keep everything the same except the covariance matrix
repeats = optimise_repeat(mean_ns, covrand*10, logL_func, logX_func, theta_actual=10, theta0=10, covinv=covinvrand, repeats=10)
optimise_repeat_stats(*repeats)

CG: 9.7404 +/- 0.1430
LS: 3.94 +/- 2.63


In [250]:
theta_actual = 10
x_actual = np.random.multivariate_normal(mean, covrand)
y = logL_func(x_actual, theta_actual)

def negative_prob(theta):
    x = logX_func(y, theta)
    return (x - mean).T @ covinvrand @ (x - mean)